In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
data = pd.read_csv('HeartDiseaseTrain-Test.csv')

In [4]:
data.head(10)

,age,sex,chest_pain_type,resting_blood_pressure,cholestoral,fasting_blood_sugar,rest_ecg,Max_heart_rate,exercise_induced_angina,oldpeak,slope,vessels_colored_by_flourosopy,thalassemia,target
0,52,Male,Typical angina,125,212,Lower than 120 mg/ml,ST-T wave abnormality,168,No,1.0,Downsloping,Two,Reversable Defect,0
1,53,Male,Typical angina,140,203,Greater than 120 mg/ml,Normal,155,Yes,3.1,Upsloping,Zero,Reversable Defect,0
2,70,Male,Typical angina,145,174,Lower than 120 mg/ml,ST-T wave abnormality,125,Yes,2.6,Upsloping,Zero,Reversable Defect,0
3,61,Male,Typical angina,148,203,Lower than 120 mg/ml,ST-T wave abnormality,161,No,0.0,Downsloping,One,Reversable Defect,0
4,62,Female,Typical angina,138,294,Greater than 120 mg/ml,ST-T wave abnormality,106,No,1.9,Flat,Three,Fixed Defect,0
5,58,Female,Typical angina,100,248,Lower than 120 mg/ml,Normal,122,No,1.0,Flat,Zero,Fixed Defect,1
6,58,Male,Typical angina,114,318,Lower than 120 mg/ml,Left ventricular hypertrophy,140,No,4.4,Upsloping,Three,Normal,0
7,55,Male,Typical angina,160,289,Lower than 120 mg/ml,Normal,145,Yes,0.8,Flat,One,Reversable Defect,0
8,46,Male,Typical angina,120,249,Lower than 120 mg/ml,Normal,144,No,0.8,Downsloping,Zero,Reversable Defect,0
9,54,Male,Typical angina,122,286,Lower than 120 mg/ml,Normal,116,Yes,3.2,Flat,Two,Fixed Defect,0


In [5]:
data.sample(10)

,age,sex,chest_pain_type,resting_blood_pressure,cholestoral,fasting_blood_sugar,rest_ecg,Max_heart_rate,exercise_induced_angina,oldpeak,slope,vessels_colored_by_flourosopy,thalassemia,target
431,65,Female,Typical angina,150,225,Lower than 120 mg/ml,Normal,114,No,1.0,Flat,Three,Reversable Defect,0
97,53,Male,Typical angina,123,282,Lower than 120 mg/ml,ST-T wave abnormality,95,Yes,2.0,Flat,Two,Reversable Defect,0
253,57,Male,Typical angina,165,289,Greater than 120 mg/ml,Normal,124,No,1.0,Flat,Three,Reversable Defect,0
865,60,Female,Asymptomatic,150,240,Lower than 120 mg/ml,ST-T wave abnormality,171,No,0.9,Downsloping,Zero,Fixed Defect,1
58,67,Male,Non-anginal pain,152,212,Lower than 120 mg/ml,Normal,150,No,0.8,Flat,Zero,Reversable Defect,0
30,44,Male,Typical angina,120,169,Lower than 120 mg/ml,ST-T wave abnormality,144,Yes,2.8,Upsloping,Zero,Normal,0
505,44,Female,Non-anginal pain,118,242,Lower than 120 mg/ml,ST-T wave abnormality,149,No,0.3,Flat,One,Fixed Defect,1
574,60,Male,Typical angina,125,258,Lower than 120 mg/ml,Normal,141,Yes,2.8,Flat,One,Reversable Defect,0
8,46,Male,Typical angina,120,249,Lower than 120 mg/ml,Normal,144,No,0.8,Downsloping,Zero,Reversable Defect,0
709,45,Male,Asymptomatic,110,264,Lower than 120 mg/ml,ST-T wave abnormality,132,No,1.2,Flat,Zero,Reversable Defect,0


In [6]:
le = LabelEncoder()

In [7]:
data['sex'] = le.fit_transform(data['sex'])
data['chest_pain_type'] = le.fit_transform(data['chest_pain_type'])
data['fasting_blood_sugar'] = le.fit_transform(data['fasting_blood_sugar'])
data['rest_ecg'] = le.fit_transform(data['rest_ecg'])
data['exercise_induced_angina'] = le.fit_transform(data['exercise_induced_angina'])
data['slope'] = le.fit_transform(data['slope'])
data['vessels_colored_by_flourosopy'] = le.fit_transform(data['vessels_colored_by_flourosopy'])
data['thalassemia'] = le.fit_transform(data['thalassemia'])

In [8]:
X = data.drop('target', axis=1)  
y = data['target']


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
model = LogisticRegression(max_iter=2000)

In [17]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=2000)

In [18]:
y_pred = model.predict(X_test)

In [19]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7902439024390244


In [20]:
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.75      0.78       102
           1       0.77      0.83      0.80       103

    accuracy                           0.79       205
   macro avg       0.79      0.79      0.79       205
weighted avg       0.79      0.79      0.79       205



In [21]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[76 26]
 [17 86]]
